# Geolocalización de antenas en el Gran Santiago.
#### Autor: Sebastián Cifuentes Carrasco

Se tiene un dataset con información de antenas ubicadas en todo Chile. El esquema del dataset es el siguiente:


| id | cid | lac | lat | lon | carrier_id |
|----|-----|-----|-----|-----|------------|

Las columnas *lat* y *lon* indican la posición geográfica de una antena dada. Lo que se hará será filtrar las antenas que pertenezcan a las comunas del Gran Santiago (pueden verse [aquí](https://es.wikipedia.org/wiki/Anexo:Comunas_de_Santiago_de_Chile#Ubicaci%C3%B3n_de_las_comunas)) a partir de su latitud y longitud para luego generar un subconjunto de antenas etiquetadas por comuna. Para esta tarea se cuenta con un [mapa](https://github.com/niclabs/maps/) de las comunas de Chile generado en formato TopoJSON y transformado al formato GeoJSON, el cual contiene polígonos georeferenciados que describen el área de las comunas antes dichas.

Para esta tarea utilizaremos las librerías [*geopandas*](http://geopandas.org/) y [*shapely*](https://shapely.readthedocs.io/en/latest/) que facilitan el manejo de los objetos de los archivos de mapas.


In [1]:
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import Point


## Geopandas dataframe
Se utiliza la librería geopandas para abrir el mapa de comunas de Chile, ya que transforma los datos dentro del archivo JSON directamente a polígonos de shapely, como puede verse en la descripción posterior del dataset.

In [2]:
#Cargamos todas las comunas y los datos de las antenas.
comunas = gpd.read_file('comunas_chile.json')
antenas = pd.read_csv('antennas.csv')

In [3]:
print("Comunas:")
print(f'{comunas.head()}\n')
print("Antenas:")
print(antenas.head())

Comunas:
      NOM_COM                  REGION     PROV  CODIGO  \
0     Ercilla  Región de La Araucanía  Malleco    9204   
1   Lonquimay  Región de La Araucanía  Malleco    9205   
2  Los Sauces  Región de La Araucanía  Malleco    9206   
3      Lumaco  Región de La Araucanía  Malleco    9207   
4       Purén  Región de La Araucanía  Malleco    9208   

                                            geometry  
0  POLYGON ((-71.98421280648006 -38.1768417919306...  
1  POLYGON ((-71.04438731634175 -38.1229627952722...  
2  POLYGON ((-72.57214016789078 -38.0229018014781...  
3  POLYGON ((-72.78671219760272 -38.1229627952722...  
4  POLYGON ((-73.01415854909737 -37.8574163117418...  

Antenas:
   id         cid         lac        lat        lon  carrier_id
0  27  2147483647  2147483647        NaN        NaN        7301
1  43          -1          -1        NaN        NaN        7301
2  82  2147483647  2147483647        NaN        NaN        7302
3  84          -1          -1        NaN      

In [4]:
#Extraemos las comunas del Gran Santiago que nos interesan.
#Todas las comunas de la ciudad de Santiago + Puente Alto, San Bdo. y  Padre Hurtado.
com_stgo = list(range(13101,13133)) + [13201, 13401, 13604]
#Extraemos los polígonos del Gran Santiago
stgo = comunas[comunas['CODIGO'].isin(com_stgo)]
#stgo.to_file('comunas_santiago.json', driver = 'GeoJSON')
stgo.head()


,NOM_COM,REGION,PROV,CODIGO,geometry
177,Independencia,Región Metropolitana de Santiago,Santiago,13108,"POLYGON ((-70.653866222266 -33.43164158623397,..."
178,La Florida,Región Metropolitana de Santiago,Santiago,13110,POLYGON ((-70.43929419255406 -33.4932175824149...
179,Lo Barnechea,Región Metropolitana de Santiago,Santiago,13115,POLYGON ((-70.51654012325037 -33.3739140898143...
180,Lo Espejo,Región Metropolitana de Santiago,Santiago,13116,POLYGON ((-70.67103198464297 -33.5124600812215...
181,Lo Prado,Región Metropolitana de Santiago,Santiago,13117,POLYGON ((-70.72682071236807 -33.4547325848018...


## Usando shapely para geolocalizar.

Para geolocalizar un punto dado, shapely considera un punto dado como Point(latitud,longitud). De otra forma, todo punto que se suponga estar dentro de un polígono dado no pertenecerá a éste. Así, utilizamos el método *contains* la cual dado un polígono y un punto, retorna si éste último está o no dentro de su contorno. De esta manera, iterando sobre cada antena, podemos ubicar a esta dentro de alguno de los polígonos de Santiago. De otra forma, se descarta el dato.

In [5]:
#Ahora geolocalizamos aquellas antenas que estén en Santiago.

antenas_stgo = []

#Iteramos sobre las antenas
for antrow in antenas.itertuples():
    if math.isnan(antrow[4]) or math.isnan(antrow[5]):
        continue
    else:
        #Generamos un punto lon/lat que identifica a una antena
        p = Point(antrow[5], antrow[4])
        #Iteramos sobre los polígonos de santiago
        for stgorow in stgo.itertuples():
            #La antena está en una comuna de Santiago?
            if stgorow[5].contains(p):
                antenas_stgo.append((antrow[1],
                                     antrow[2],
                                     antrow[3],
                                     antrow[4],
                                     antrow[5],
                                     antrow[6],
                                     stgorow[1]))
                break #Salimos pues ya localizamos la antena en alguna comuna y no es necesario chequear el resto

In [6]:
col_antenas_stgo = ['id', 'cid', 'lac', 'lat', 'lon', 'carrier_id', 'comuna']
df_antenas_stgo = pd.DataFrame(antenas_stgo, columns = col_antenas_stgo)
#df_antenas_stgo.to_csv('antenas_santiago.csv', index = False)
print(df_antenas_stgo.head())
print(df_antenas_stgo.shape)


   id       cid    lac        lat        lon  carrier_id       comuna
0   2   1223713   1330 -33.471877 -70.581209        7302        Ñuñoa
1   4   1196265  52025 -33.496543 -70.746325        7302        Maipú
2   6  87053737  13510 -33.533728 -70.663414        7301  La Cisterna
3   8  87082065  13510 -33.537654 -70.663381        7301  La Cisterna
4   9  86991430  13622 -33.536916 -70.590676        7301   La Florida
(51487, 7)


## Antenas ubicadas!

Con las antenas ya ubicadas por comuna, podemos hacer un análisis simple sobre su distribución en Santiago, viendo la cantidad total de antenas que cada comuna contiene.

In [7]:
#Vamos a hacer un análisis simple sobre las antenas, para ver su distribución por comunas
df_antenas_stgo.groupby(['comuna']).size()

comuna
Cerrillos               826
Cerro Navia             260
Conchalí                575
El Bosque               517
Estación Central       1495
Huechuraba              843
Independencia           689
La Cisterna             739
La Florida             2659
La Granja               472
La Pintana              536
La Reina               1085
Las Condes             4401
Lo Barnechea            827
Lo Espejo               400
Lo Prado                405
Macul                  1601
Maipú                  2750
Padre Hurtado           300
Pedro Aguirre Cerda     459
Peñalolén              1304
Providencia            4525
Pudahuel               1390
Puente Alto            1955
Quilicura               963
Quinta Normal           566
Recoleta               1257
Renca                   633
San Bernardo           1543
San Joaquín             833
San Miguel              949
San Ramón               312
Santiago               8989
Vitacura               1583
Ñuñoa                  2846
dtype: int64